import pandas as pd
import numpy as np
import math
import os
import warnings
print('Modulos importados')# Algoritmos auxiliares

En este script se plantea mostrar cómo funcionan algortimos encontrados en internet. Esto, debido a que hay veces en las que son más eficientes que los algoritmos propios, por lo que solo pondré los que son más eficientes que los propios y que haya encontrado.

In [3]:
import pandas as pd
import numpy as np
import math
import os
import warnings
print('Modulos importados')

Modulos importados


In [4]:
# Directorio de trabajo
print("Directorio de trabajo previo: " + str(os.getcwd()))
# Cambiemoslo
os.chdir('/home/usuario/Documentos/Github/Proyectos/MLB_HN/')

Directorio de trabajo previo: /home/usuario/Documentos/Github/Proyectos/MLB_HN/Programming_test


### Valores únicos por fila

Obtengamos los valores únicos por fila a través de distintos métodos

In [26]:
lst = [[12, 12, 25], [34, 3, 30],
       [26, 26, 26], [np.nan, 1, 22], 
       [np.nan, np.nan, np.nan]]

In [27]:
df = pd.DataFrame(lst, columns =['FName', 'LName', 'Age'], dtype = float)
df

,FName,LName,Age
0,12.0,12.0,25.0
1,34.0,3.0,30.0
2,26.0,26.0,26.0
3,NaN,1.0,22.0
4,NaN,NaN,NaN


In [28]:
list(map(set,df.values))

[{12.0, 25.0}, {3.0, 30.0, 34.0}, {26.0}, {nan, 1.0, 22.0}, {nan, nan, nan}]

In [29]:
df.stack().groupby(level=0).apply(lambda x: x.unique().tolist())

0         [12.0, 25.0]
1    [34.0, 3.0, 30.0]
2               [26.0]
3          [1.0, 22.0]
dtype: object

El anterior es el mejor método

In [30]:
df.T.agg([pd.unique])

,0,1,2,3,4
unique,"[12.0, 25.0]","[34.0, 3.0, 30.0]",[26.0],"[nan, 1.0, 22.0]",[nan]


### Contar valores únicos por fila

In [31]:
df = pd.DataFrame(lst, columns =['FName', 'LName', 'Age'], dtype = float)

df['num_uniq_1'] = [len(set(v[pd.notna(v)].tolist())) for v in df.values]
df

,FName,LName,Age,num_uniq_1
0,12.0,12.0,25.0,2
1,34.0,3.0,30.0,3
2,26.0,26.0,26.0,1
3,NaN,1.0,22.0,2
4,NaN,NaN,NaN,0


In [32]:
df = pd.DataFrame(lst, columns =['FName', 'LName', 'Age'], dtype = float)

# df.join(df.stack().groupby(level=0).nunique().to_frame('num_uniq'))
df['num_uniq_2'] = df.stack().groupby(level=0).nunique()
df

,FName,LName,Age,num_uniq_2
0,12.0,12.0,25.0,2.0
1,34.0,3.0,30.0,3.0
2,26.0,26.0,26.0,1.0
3,NaN,1.0,22.0,2.0
4,NaN,NaN,NaN,NaN


In [33]:
df = pd.DataFrame(lst, columns =['FName', 'LName', 'Age'], dtype = float)

df['num_uniq_3'] = df.apply(pd.Series.nunique, axis=1)
df

,FName,LName,Age,num_uniq_3
0,12.0,12.0,25.0,2
1,34.0,3.0,30.0,3
2,26.0,26.0,26.0,1
3,NaN,1.0,22.0,2
4,NaN,NaN,NaN,0


### Buscar un valor entre dos dataframes

Nos interesa hallar la fila en la que dos dataframes tienen un mismo valor con respecto a una columna

In [2]:
df1 = pd.DataFrame({'Time': ['8a', '10p'], 'V1': [1, 2], 'V2': [3, 4]})
df2 = pd.DataFrame({'fn': ['8.txt', '10.txt'], 'V1': [3, 2], 'V2': [3, 4]})

Primera solución

In [3]:
df1.merge(df2, on=['V1', 'V2'], how='outer').dropna()

,Time,V1,V2,fn
1,10p,2,4,10.txt


In [8]:
df2.merge(df1,how="inner",on=["V1","V2"])[["fn","Time"]]

,fn,Time
0,10.txt,10p


In [9]:
pd.concat([df1, df2.reindex(df1.index)], axis=1)

,Time,V1,V2,fn,V1,V2
0,8a,1,3,8.txt,3,3
1,10p,2,4,10.txt,2,4


Segunda solución y óptima para los propósitos

In [4]:
for row in range(len(df1)):
    if (df1.iloc[row,1:] == df2.iloc[row,1:]).all() == True: 
        print(df1.iloc[row], df2.iloc[row])

Time    10p
V1        2
V2        4
Name: 1, dtype: object fn    10.txt
V1         2
V2         4
Name: 1, dtype: object


In [6]:
client = boto3.client('s3')

obj = client.get_object(Bucket='', Key='')
data = obj['Body'].read()
df1 = pd.read_excel(io.BytesIO(data), sheet_name='0')
df2 = pd.read_excel(io.BytesIO(data), sheet_name='1')

head = df2.columns[0]
print(head)

data = df1.iloc[[8],[0]].values[0]
print(data)

print(df2)
df2.columns = df2.iloc[0]
df2 = df2.drop(labels=0, axis=0)
df2['Head'] = head
df2['ID'] = pd.Series([data,data])

print(df2)
df2.to_csv('test.csv',index=False)

NameError: name 'boto3' is not defined

## Merge de bases de datos de forma condicional

Lo que se hará es crear un algoritmo que filtre las bases de datos cuyos años de contrato restante sean mayores a uno y luego mostrarlos en bucle hasta que dicha cantidad sea cero

In [5]:
# Auxiliares:
free_agents = 'Data/Free_Agents/Free_Agents_'
hitting = 'Data/Statistics/Hitting/hitting_'
pitching = 'Data/Statistics/Pitching/pitching_'
salary = 'Data/Salary/Salary_'
teams = 'ETL_data/Teams/free_agents_team_'
csv = '.csv'
period = 11
# Originales:
df_free_agents = [None]*period
df_hitting = [None]*period
df_pitching = [None]*period
df_salary = [None]*period
df_teams = [None]*period
# Copias:
df_free_agents_copy = [None]*period
df_hitting_copy = [None]*period
df_pitching_copy = [None]*period
df_salary_copy = [None]*period
df_teams_copy = [None]*period
# Producto final:
df_pitchers = [None]*period
df_hitters = [None]*period
df_pitchers_free_agents = [None]*period
df_hitters_free_agents = [None]*period
df_pitchers_no_free_agents = [None]*period
df_hitters_no_free_agents = [None]*period

In [6]:
for i in range(0,period):    
    df_free_agents[i] = pd.read_csv(free_agents + str(2011 + i) + csv)
    df_hitting[i] = pd.read_csv(hitting + str(2011 + i) + csv)
    df_pitching[i] = pd.read_csv(pitching + str(2011 + i) + csv)
    df_salary[i] = pd.read_csv(salary + str(2011 + i) + csv)
    df_teams[i] = pd.read_csv(teams + str(2011 + i) + csv)
    
    df_free_agents_copy[i] = df_free_agents[i].copy()
    df_hitting_copy[i] = df_hitting[i].copy()
    df_pitching_copy[i] = df_pitching[i].copy()
    df_salary_copy[i] = df_salary[i].copy()
    df_teams_copy[i] = pd.read_csv(teams + str(2011 + i) + csv)

Veamos el contenido de las bases de datos de los agentes libres

In [5]:
df_free_agents_copy[4].head()

,Rank,Player,Year,Pos,Status,Team From,Team From To,YRS,Value,AAV
0,1,Max Scherzer,2015,SP,UFA,DET,WSH,7,"$210,000,000","$30,000,000"
1,2,Jon Lester,2015,SP,UFA,OAK,CHC,6,"$155,000,000","$25,833,333"
2,3,Pablo Sandoval,2015,3B,UFA,SF,BOS,5,"$95,000,000","$19,000,000"
3,4,Hanley Ramirez,2015,1B,UFA,LAD,BOS,4,"$88,000,000","$22,000,000"
4,5,Russell Martin,2015,C,UFA,PIT,TOR,5,"$82,000,000","$16,400,000"


Filtremos aquellos que tienen más de un año de contrato

In [6]:
df_free_agents_copy[4][df_free_agents_copy[4]['YRS'] > 1]

,Rank,Player,Year,Pos,Status,Team From,Team From To,YRS,Value,AAV
0,1,Max Scherzer,2015,SP,UFA,DET,WSH,7,"$210,000,000","$30,000,000"
1,2,Jon Lester,2015,SP,UFA,OAK,CHC,6,"$155,000,000","$25,833,333"
2,3,Pablo Sandoval,2015,3B,UFA,SF,BOS,5,"$95,000,000","$19,000,000"
3,4,Hanley Ramirez,2015,1B,UFA,LAD,BOS,4,"$88,000,000","$22,000,000"
4,5,Russell Martin,2015,C,UFA,PIT,TOR,5,"$82,000,000","$16,400,000"
5,6,James Shields,2015,SP,UFA,KC,SD,4,"$75,000,000","$18,750,000"
6,7,Nelson Cruz,2015,DH,UFA,BAL,SEA,4,"$57,000,000","$14,250,000"
7,8,Ervin Santana,2015,SP,UFA,ATL,MIN,4,"$55,000,000","$13,750,000"
8,9,Chase Headley,2015,3B,UFA,NYY,NYY,4,"$52,000,000","$13,000,000"
9,10,Brandon McCarthy,2015,SP,UFA,NYY,LAD,4,"$48,000,000","$12,000,000"


Ahora, haremos un bucle para mostrar progrsivamente los jugadores con contratos que duren más y más años.

In [19]:
period = 11

Veamos la cantidad de datos que se añadirían a cada base de datos anual con respecto a la base de datos de algún año previo

In [9]:
for year in range(0,period):
    
    max_year_contract = max(df_free_agents_copy[year]['YRS'])
    print("Max year = " + str(max_year_contract))
    print('Year = ' + str(2011 + year))
    
    for incremento in range(0,max_year_contract - 1):
        merge = df_free_agents_copy[year][df_free_agents_copy[year]['YRS'] > 1 + incremento]
        
        print("Año base de datos = " + str(2011 + 1 + incremento))
        print("Len = " +str(len(merge)))
    print("\n")
    print("\n")

Max year = 1
Year = 2011




Max year = 10
Year = 2012
Año base de datos = 2012
Len = 30
Año base de datos = 2013
Len = 10
Año base de datos = 2014
Len = 6
Año base de datos = 2015
Len = 4
Año base de datos = 2016
Len = 3
Año base de datos = 2017
Len = 2
Año base de datos = 2018
Len = 2
Año base de datos = 2019
Len = 2
Año base de datos = 2020
Len = 1




Max year = 6
Year = 2013
Año base de datos = 2012
Len = 42
Año base de datos = 2013
Len = 16
Año base de datos = 2014
Len = 6
Año base de datos = 2015
Len = 4
Año base de datos = 2016
Len = 1




Max year = 10
Year = 2014
Año base de datos = 2012
Len = 50
Año base de datos = 2013
Len = 20
Año base de datos = 2014
Len = 10
Año base de datos = 2015
Len = 4
Año base de datos = 2016
Len = 3
Año base de datos = 2017
Len = 3
Año base de datos = 2018
Len = 1
Año base de datos = 2019
Len = 1
Año base de datos = 2020
Len = 1




Max year = 7
Year = 2015
Año base de datos = 2012
Len = 35
Año base de datos = 2013
Len = 19
Año base de datos = 201

Algo que notamos es que hay contratos que se extienden a periodos fuera del año máximo de análisis, 2021. Debido a esto, tendremos que restringir el año a indexar.

In [15]:
for year in range(0,period):
    
    max_year_contract = max(df_free_agents_copy[year]['YRS'])
    print("Max year = " + str(max_year_contract))
    print('Year = ' + str(2011 + year))
    
    for incremento in range(0,max_year_contract - 1):
        diff_t = 1 + incremento
        real_year = 2011 + diff_t
        year_bound = 2021

        if real_year <= year_bound:
            merge = df_free_agents_copy[year][df_free_agents_copy[year]['YRS'] > diff_t]

            print("Año base de datos = " + str(2011 + diff_t))
            print("Len = " +str(len(merge)))
    print("\n")
    print("\n")

Max year = 1
Year = 2011




Max year = 10
Year = 2012
Año base de datos = 2012
Len = 30
Año base de datos = 2013
Len = 10
Año base de datos = 2014
Len = 6
Año base de datos = 2015
Len = 4
Año base de datos = 2016
Len = 3
Año base de datos = 2017
Len = 2
Año base de datos = 2018
Len = 2
Año base de datos = 2019
Len = 2
Año base de datos = 2020
Len = 1




Max year = 10
Year = 2013
Año base de datos = 2012
Len = 102
Año base de datos = 2013
Len = 36
Año base de datos = 2014
Len = 18
Año base de datos = 2015
Len = 12
Año base de datos = 2016
Len = 7
Año base de datos = 2017
Len = 4
Año base de datos = 2018
Len = 4
Año base de datos = 2019
Len = 4
Año base de datos = 2020
Len = 2




Max year = 10
Year = 2014
Año base de datos = 2012
Len = 244
Año base de datos = 2013
Len = 102
Año base de datos = 2014
Len = 52
Año base de datos = 2015
Len = 32
Año base de datos = 2016
Len = 20
Año base de datos = 2017
Len = 13
Año base de datos = 2018
Len = 11
Año base de datos = 2019
Len = 11
Año base d

Un problema que se tiene que tomar en cuenta es que si añadimos los datos a bases de datos posteriores a un año, estas se añadirán otra vez. Debido a esto, es más sencillo controlar esto si guardamos los cambios debido a un dataframe en distintos dataframes.

In [7]:
period_t = period - 1
df_contracts = [None]*(period_t)

In [8]:
len(df_contracts)

10

Solo serán 10 años los que tenemos que tomar en cuenta puesto que los contratos de la base de datos del 2021 no se añadirán a ninguna otra base de datos. Por otro lado, como en el año 2011 no hay ninguna contribución para contratos futuros, omitiremos ese año

In [9]:
for years in range(1,period_t):
    diff_remain = period_t - years
    df_contracts[years] = [None]*diff_remain
    
    print("Year = " + str(2011 + years))
    print("Length = " + str(len(df_contracts[years])))
    for sub_year in range(0,len(df_contracts[years])):
        print(df_contracts[years][sub_year])
    
    print("")

Year = 2012
Length = 9
None
None
None
None
None
None
None
None
None

Year = 2013
Length = 8
None
None
None
None
None
None
None
None

Year = 2014
Length = 7
None
None
None
None
None
None
None

Year = 2015
Length = 6
None
None
None
None
None
None

Year = 2016
Length = 5
None
None
None
None
None

Year = 2017
Length = 4
None
None
None
None

Year = 2018
Length = 3
None
None
None

Year = 2019
Length = 2
None
None

Year = 2020
Length = 1
None



Guardemos en cada una de las entradas correspondientes los distintos registros de jugadores con contratos para más de una temporada.

In [15]:
for year in range(1,period_t):
    
    max_year_contract = max(df_free_agents_copy[year]['YRS'])
    years = max_year_contract - 1
    df_contracts[year] = [None]*years
    
    print("Max year = " + str(max_year_contract))
    print('Year = ' + str(2011 + year))
    
    for incremento in range(0,years):
        diff_t = 1 + incremento
        real_year = 2011 + year + diff_t
        year_bound = 2021

        if real_year <= year_bound:
            df_contracts[year][incremento] = df_free_agents_copy[year][df_free_agents_copy[year]['YRS'] > diff_t]
            
            print("Año base de datos = " + str(real_year))
    print("\n")
    print("\n")

Max year = 10
Year = 2012
Año base de datos = 2013
Año base de datos = 2014
Año base de datos = 2015
Año base de datos = 2016
Año base de datos = 2017
Año base de datos = 2018
Año base de datos = 2019
Año base de datos = 2020
Año base de datos = 2021




Max year = 6
Year = 2013
Año base de datos = 2014
Año base de datos = 2015
Año base de datos = 2016
Año base de datos = 2017
Año base de datos = 2018




Max year = 10
Year = 2014
Año base de datos = 2015
Año base de datos = 2016
Año base de datos = 2017
Año base de datos = 2018
Año base de datos = 2019
Año base de datos = 2020
Año base de datos = 2021




Max year = 7
Year = 2015
Año base de datos = 2016
Año base de datos = 2017
Año base de datos = 2018
Año base de datos = 2019
Año base de datos = 2020
Año base de datos = 2021




Max year = 8
Year = 2016
Año base de datos = 2017
Año base de datos = 2018
Año base de datos = 2019
Año base de datos = 2020
Año base de datos = 2021




Max year = 5
Year = 2017
Año base de datos = 2018
Año

Veamos algunos ejemplos

In [16]:
df_contracts[1][2]

,Rank,Player,Year,Pos,Status,Team From,Team From To,YRS,Value,AAV
0,1,Albert Pujols,2012,DH,UFA,STL,LAA,10,"$240,000,000","$24,000,000"
1,2,Prince Fielder,2012,DH,UFA,MIL,DET,9,"$214,000,000","$23,777,778"
2,3,Jose Reyes,2012,SS,UFA,NYM,MIA,6,"$106,000,000","$17,666,667"
3,4,C.J. Wilson,2012,SP,UFA,TEX,LAA,5,"$77,500,000","$15,500,000"
4,5,Mark Buehrle,2012,SP,UFA,CHW,MIA,4,"$58,000,000","$14,500,000"
5,6,Jonathan Papelbon,2012,RP,UFA,BOS,PHI,4,"$50,000,000","$12,500,000"


In [17]:
df_contracts[3][5]

,Rank,Player,Year,Pos,Status,Team From,Team From To,YRS,Value,AAV
0,1,Robinson Cano,2014,2B,UFA,NYY,SEA,10,"$240,000,000","$24,000,000"
1,2,Jacoby Ellsbury,2014,CF,UFA,BOS,NYY,7,"$153,000,000","$21,857,143"
2,3,Shin-Soo Choo,2014,DH,UFA,CIN,TEX,7,"$130,000,000","$18,571,429"


Concatenemos estas bases de datos con las de los agentes libres

In [18]:
for year in range(1,period_t):
    
    years = len(df_contracts[year])
    
    print("Max year = " + str(max_year_contract))
    print('Year = ' + str(2011 + year))
    
    for incremento in range(0,years):
        diff_t = 1 + incremento
        real_year = 2011 + year + diff_t
        year_bound = 2021

        if real_year <= year_bound:
            frames = [df_free_agents_copy[year + diff_t], df_contracts[year][incremento]]
            
            df_free_agents_copy[year + diff_t] = pd.concat(frames)

            df_free_agents_copy[year + diff_t].reset_index(drop = True, inplace = True)
            df_free_agents_copy[year + diff_t] = df_free_agents_copy[year + diff_t].sort_values(by = 'Rank', ascending = True)
            
            print("Año base de datos = " + str(real_year))
    print("\n")
    print("\n")

Max year = 9
Year = 2012
Año base de datos = 2013
Año base de datos = 2014
Año base de datos = 2015
Año base de datos = 2016
Año base de datos = 2017
Año base de datos = 2018
Año base de datos = 2019
Año base de datos = 2020
Año base de datos = 2021




Max year = 9
Year = 2013
Año base de datos = 2014
Año base de datos = 2015
Año base de datos = 2016
Año base de datos = 2017
Año base de datos = 2018




Max year = 9
Year = 2014
Año base de datos = 2015
Año base de datos = 2016
Año base de datos = 2017
Año base de datos = 2018
Año base de datos = 2019
Año base de datos = 2020
Año base de datos = 2021




Max year = 9
Year = 2015
Año base de datos = 2016
Año base de datos = 2017
Año base de datos = 2018
Año base de datos = 2019
Año base de datos = 2020
Año base de datos = 2021




Max year = 9
Year = 2016
Año base de datos = 2017
Año base de datos = 2018
Año base de datos = 2019
Año base de datos = 2020
Año base de datos = 2021




Max year = 9
Year = 2017
Año base de datos = 2018
Año b

Verifiquemos el contenidos de las bases de datos de los agentes

In [19]:
for year in range(0,period):
    print(df_free_agents_copy[year].shape)

(1, 10)
(108, 10)
(243, 10)
(260, 10)
(293, 10)
(306, 10)
(182, 10)
(174, 10)
(189, 10)
(192, 10)
(203, 10)


In [20]:
for year in range(0,period):
    print(df_free_agents_copy[year].loc[0:5])

   Rank      Player  Year Pos Status Team From Team From To  YRS     Value  \
0     1  Wil Nieves  2011   C    UFA       WSH          MIL    1  $775,000   

        AAV  
0  $775,000  
   Rank             Player  Year Pos Status Team From Team From To  YRS  \
0     1      Albert Pujols  2012  DH    UFA       STL          LAA   10   
1     2     Prince Fielder  2012  DH    UFA       MIL          DET    9   
2     3         Jose Reyes  2012  SS    UFA       NYM          MIA    6   
3     4        C.J. Wilson  2012  SP    UFA       TEX          LAA    5   
4     5       Mark Buehrle  2012  SP    UFA       CHW          MIA    4   
5     6  Jonathan Papelbon  2012  RP    UFA       BOS          PHI    4   

          Value          AAV  
0  $240,000,000  $24,000,000  
1  $214,000,000  $23,777,778  
2  $106,000,000  $17,666,667  
3   $77,500,000  $15,500,000  
4   $58,000,000  $14,500,000  
5   $50,000,000  $12,500,000  
     Rank             Player  Year Pos Status Team From Team From To  YR